In [13]:
import pandas as pd
import numpy as np

## Load wind power data

In [24]:
df_wind_power = pd.read_csv('../data/wind/wind_power_train.csv',
                            header=0,
                            index_col=0,
                            parse_dates=True)

In [25]:
df_wind_meteo = pd.read_csv('../data/wind/wind_meteo_train.csv',
                            header=[0, 1],
                            skiprows=[2],
                            index_col=0,
                            parse_dates=True)

In [26]:
df_wind_power.head()

,1,2,3,4,5,6,7,8,9,10
TIMESTAMP,,,,,,,,,,
2012-01-01 01:00:00,0.000000,0.596273,0.425465,0.378229,0.273678,0.268101,0.000000,0.000000,0.0,0.594361
2012-01-01 02:00:00,0.054879,0.411180,0.363851,0.063012,0.086796,0.034542,0.014708,0.014954,0.0,0.569679
2012-01-01 03:00:00,0.110234,0.167243,0.297129,0.036704,0.006811,0.020621,0.035098,0.071785,0.0,0.330539
2012-01-01 04:00:00,0.165116,0.037326,0.235438,0.034342,0.018646,0.001867,0.045055,0.066796,0.0,0.211308
2012-01-01 05:00:00,0.156940,0.051206,0.120904,0.033554,0.034812,0.015174,0.050160,0.034797,0.0,0.172140


## Load solar power data

In [28]:
df_solar_power = pd.read_csv('../data/solar/solar_power_train.csv',
                             header=0,
                             index_col=0,
                             parse_dates=True)

df_solar_meteo = pd.read_csv('../data/solar/solar_meteo_train.csv',
                             header=[0, 1],
                             skiprows=[2],
                             index_col=0,
                             parse_dates=True)

In [30]:
df_solar_power.head()

,1,2,3
TIMESTAMP,,,
2012-04-01 01:00:00,0.754103,0.647328,0.799000
2012-04-01 02:00:00,0.555000,0.621822,0.817500
2012-04-01 03:00:00,0.438397,0.472692,0.571500
2012-04-01 04:00:00,0.145449,0.098866,0.063775
2012-04-01 05:00:00,0.111987,0.104393,0.103700


In [31]:
df_solar_meteo.head()

ZONEID,1,2,3,1,2,3,1,2,3,1,...,3,1,2,3,1,2,3,1,2,3
,VAR134,VAR134,VAR134,VAR157,VAR157,VAR157,VAR164,VAR164,VAR164,VAR165,...,VAR178,VAR228,VAR228,VAR228,VAR78,VAR78,VAR78,VAR79,VAR79,VAR79
2012-04-01 01:00:00,94843.6250,94290.6250,92667.6250,60.221909,57.374252,57.003159,0.244601,0.190966,0.238147,1.039334,...,2901861.0,0.000000,0.000000,0.000000,0.001967,0.001479,0.003531,0.003609,0.005444,0.024673
2012-04-01 02:00:00,94757.9375,94217.6875,92622.6875,54.678604,57.129776,64.960831,0.457138,0.517135,0.561615,2.482865,...,5980930.0,0.000000,0.000000,0.000774,0.005524,0.018304,0.027482,0.033575,0.101553,0.140474
2012-04-01 03:00:00,94732.8125,94201.0625,92598.8125,61.294891,63.076141,66.795868,0.771429,0.705222,0.661612,3.339867,...,8822632.0,0.001341,0.001256,0.001671,0.030113,0.035255,0.032356,0.132009,0.112935,0.136617
2012-04-01 04:00:00,94704.0625,94156.0625,92542.0625,67.775284,62.975479,65.177628,0.965866,0.993363,0.941559,3.106102,...,11178847.0,0.002501,0.001726,0.002084,0.057167,0.064514,0.031952,0.110645,0.107761,0.097370
2012-04-01 05:00:00,94675.0000,94124.2500,92508.0000,70.172989,65.053848,65.744278,0.944669,0.937604,0.944684,2.601146,...,13079958.0,0.003331,0.002093,0.002534,0.051027,0.059495,0.048494,0.189560,0.157593,0.163773
